# Pystencils 3D Star Stencil Volume Comparison - AMD Version

3D7pt stencil with different block sizes

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import sys 
sys.path.append('../../pystencils')
sys.path.append('../pystencils')
sys.path.append('../measutils')
sys.path.append('../warpspeed')


%load_ext autoreload
%autoreload 1

%aimport predict
%aimport griditeration
%aimport volumes_isl
%aimport pystencils.astnodes
%aimport plot_utils
%aimport predict_metrics



In [ ]:
import cProfile
import re


import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sys
from subprocess import run, PIPE


import pystencils as ps
from pystencils.slicing import add_ghost_layers, make_slice, remove_ghost_layers
from pystencils.display_utils import show_code, get_code_str
from griditeration import *
from predict_metrics import *
from volumes_isl import *
from plot_utils import *
from measured_metrics import MeasuredMetrics, ResultComparer

from pystencils_stencil_utils import getStencilKernel
from pystencilswarpspeedkernel import PyStencilsWarpSpeedKernel


import sympy as sp


import timeit

from meas_db import MeasDB

In [ ]:
meas_db_amd = MeasDB("3dstencils2_AMD.db")
meas_db_nv = MeasDB("3dstencils2.db")
#meas_db.clearDB()

In [ ]:
mi210Values = dict()
a100Values = dict()

def getBlockSizes(threadCounts):
    blockSizes = []
    for xblock in [ 8, 16, 32, 64, 128, 256, 512, 1024]:
        for yblock in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512]:
            for zblock in [1, 2, 4, 8, 16, 32, 64]:
                if xblock*yblock*zblock in threadCounts:
                    blockSizes.append((xblock, yblock, zblock))
    return blockSizes

for fieldSize in [(640, 512,200), (1024, 512, 200), (1023, 512, 200)]:
    for r in [0]:
        for blockingFactors in [(1,1,1)]: #, (1,8,1), (2,2,2)]:
            for blockSize in getBlockSizes([1024]):
                t1 = time.process_time()

                key = (r, *blockSize, blockingFactors)

                lc1, basic1, meas1 = meas_db_amd.getEntry(r, blockSize, blockingFactors, fieldSize, DeviceMI210())        
                lc2, basic2, meas2 = meas_db_nv.getEntry(r, blockSize, blockingFactors, fieldSize, DeviceAmpereA100_80GB())          

                if meas1 is not None:
                    print(meas1)
                    mi210Values[key] = meas1
                    
                    
                if meas2 is not None:        
                    a100Values[key] = meas2
                    print(meas2)

                print()

In [ ]:
volumeScatterPlot([(k[1:4], mi210Values[k].memLoad + mi210Values[k].memStore, a100Values[k].memLoad +  a100Values[k].memStore, k[4]) for k in mi210Values], "Memory Volumes MI210 vs A100 r" + str(r) )
volumeScatterPlot([(k[1:4], mi210Values[k].L2Load + mi210Values[k].L2Store, a100Values[k].L2Load_tex + a100Values[k].L2Store, k[4]) for k in mi210Values], "L2 Volumes MI210 vs A100 r" + str(r) )
volumeScatterPlot([(k[1:4], mi210Values[k].L1Wavefronts_TA, a100Values[k].L1Wavefronts, k[4]) for k in mi210Values], "L1 wavefronts MI210 vs A100 r" + str(r) )
volumeScatterPlot([(k[1:4], mi210Values[k].lups, a100Values[k].lups, k[4]) for k in mi210Values], "perf MI210 vs A100 r" + str(r) )